In [1]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_regimes(model_name, root, csv_file, regimes=['0.1', '0.2', '0.5', '1.0']):
    root = Path(root)

    metrics = []
    for run in list(root.glob("run-*")):
        if root.name.rsplit("-", 1)[1] in regimes:
            metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>Evaluation - Searching temperature hyperparameter</h1>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

<h2>GlaS Dataset</h2>

In [12]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
}

In [13]:
# Collect predictions scanning runs
#predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [14]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,51.19,35.42,98.00,15.93
1,H-UNet-BASE-SWTA,0,5.0,1.0,57.13,41.30,75.36,11.96
2,H-UNet-BASE-SWTA,0,10.0,1.0,58.14,42.35,72.67,11.51
3,H-UNet-BASE-SWTA,0,20.0,1.0,58.31,42.53,72.16,11.42
4,H-UNet-BASE-SWTA,0,50.0,1.0,58.17,42.38,72.53,11.48
5,H-UNet-BASE-SWTA,0,100.0,1.0,58.39,42.61,72.00,11.39
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,53.70,37.84,86.94,13.96
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,61.60,46.11,65.87,10.31
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,60.43,44.82,63.41,10.00
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,61.81,46.37,64.47,10.11


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   1.0      1.0     51.19 NaN   35.42 NaN              98.00   
                   5.0      1.0     57.13 NaN   41.30 NaN              75.36   
                   10.0     1.0     58.14 NaN   42.35 NaN              72.67   
                   20.0     1.0     58.31 NaN   42.53 NaN              72.16   
                   50.0     1.0     58.17 NaN   42.38 NaN              72.53   
                   100.0    1.0     58.39 NaN   42.61 NaN              72.00   
H-UNet-BASE-SWTA-T 1.0      1.0     53.70 NaN   37.84 NaN              86.94   
                   5.0      1.0     61.60 NaN   46.11 NaN              65.87   
                   10.0     1.0     60.43 NaN   44.82 NaN              63.41   
                   20.0     1.0     61.81 NaN   46.37 NaN              64.47   
                   50.0     1.0     62.01 NaN   46.58 NaN              64.98   
                   100.0    1.0     62.08 NaN   46.67 NaN              64.84   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0    NaN                    15.93 NaN  
                   5.0      1.0    NaN                    11.96 NaN  
                   10.0     1.0    NaN                    11.51 NaN  
                   20.0     1.0    NaN                    11.42 NaN  
                   50.0     1.0    NaN                    11.48 NaN  
                   100.0    1.0    NaN                    11.39 NaN  
H-UNet-BASE-SWTA-T 1.0      1.0    NaN                    13.96 NaN  
                   5.0      1.0    NaN                    10.31 NaN  
                   10.0     1.0    NaN                    10.00 NaN  
                   20.0     1.0    NaN                    10.11 NaN  
                   50.0     1.0    NaN                    10.17 NaN  
                   100.0    1.0    NaN                    10.15 NaN

In [12]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromBestDice')),
    'H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromBestDice')),
    'H-UNet-BASE-SWTA-FT-FROM-LAST': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromLast')),
    'H-UNet-BASE-SWTA-T-FT-FROM-LAST': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromLast')),
    #'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ft')),
}

In [13]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [14]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE,0,1.0,0.2,75.52,61.71,23.86,3.63
1,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE,0,5.0,0.2,79.31,66.84,27.46,3.98
2,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE,0,10.0,0.2,72.62,57.85,29.93,4.05
3,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE,0,20.0,0.2,77.55,64.08,25.19,3.29
4,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE,0,50.0,0.2,75.98,61.95,21.13,2.88
...,...,...,...,...,...,...,...,...
134,H-UNet-BASE-SWTA-T-FT-FROM-LAST,7,5.0,0.2,77.73,65.04,37.46,5.46
135,H-UNet-BASE-SWTA-T-FT-FROM-LAST,8,1.0,0.2,71.79,57.18,34.11,5.15
136,H-UNet-BASE-SWTA-T-FT-FROM-LAST,8,5.0,0.2,77.93,64.84,24.36,3.23
137,H-UNet-BASE-SWTA-T-FT-FROM-LAST,9,1.0,0.2,70.10,54.93,32.55,4.82


Dice            \
                                                           mean       std   
Model                                Inv Temp Regime                        
H-UNet-BASE-SWTA-FT-FROM-BEST-DICE   1.0      0.2     73.972000  4.027297   
                                     5.0      0.2     76.712000  3.251064   
                                     10.0     0.2     74.456000  2.125001   
                                     20.0     0.2     75.726000  1.694397   
                                     50.0     0.2     76.638000  2.727567   
                                     100.0    0.2     76.220000  4.184704   
H-UNet-BASE-SWTA-FT-FROM-LAST        1.0      0.2     71.587000  1.982995   
                                     5.0      0.2     75.618889  2.087220   
                                     10.0     0.2     75.016000  2.807121   
                                     20.0     0.2     75.870000  3.389211   
                                     50.0     0.2     77.704000  1.469874   
                                     100.0    0.2     76.816000  3.772497   
H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE 1.0      0.2     74.248000  3.073438   
                                     5.0      0.2     75.744000  1.746319   
                                     10.0     0.2     75.974000  2.488791   
                                     20.0     0.2     77.738000  2.875842   
                                     50.0     0.2     75.836000  2.129878   
                                     100.0    0.2     76.568000  3.254431   
H-UNet-BASE-SWTA-T-FT-FROM-LAST      1.0      0.2     71.575000  2.215698   
                                     5.0      0.2     76.113000  1.229472   
                                     10.0     0.2     76.532000  3.251687   
                                     20.0     0.2     76.722000  1.406617   
                                     50.0     0.2     76.680000  2.810952   
                                     100.0    0.2     77.536000  1.284671   

                                                        Jaccard            \
                                                           mean       std   
Model                                Inv Temp Regime                        
H-UNet-BASE-SWTA-FT-FROM-BEST-DICE   1.0      0.2     59.822000  5.028923   
                                     5.0      0.2     63.252000  4.308859   
                                     10.0     0.2     60.496000  2.699478   
                                     20.0     0.2     61.986000  2.154491   
                                     50.0     0.2     63.198000  3.382716   
                                     100.0    0.2     62.784000  5.371730   
H-UNet-BASE-SWTA-FT-FROM-LAST        1.0      0.2     56.772000  2.540375   
                                     5.0      0.2     61.893333  2.689507   
                                     10.0     0.2     61.000000  3.570448   
                                     20.0     0.2     62.192000  4.150051   
                                     50.0     0.2     64.572000  1.894695   
                                     100.0    0.2     63.474000  4.915611   
H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE 1.0      0.2     60.106000  3.772722   
                                     5.0      0.2     62.078000  2.331195   
                                     10.0     0.2     62.306000  3.184514   
                                     20.0     0.2     64.604000  3.846184   
                                     50.0     0.2     62.096000  2.672570   
                                     100.0    0.2     63.064000  4.289129   
H-UNet-BASE-SWTA-T-FT-FROM-LAST      1.0      0.2     56.807000  2.860567   
                                     5.0      0.2     62.547000  1.616313   
                                     10.0     0.2     63.250000  4.087707   
                                     20.0     0.2     63.336000  1.721941   
                                     50.0     0.2     

<h1>Evaluation - Hebbian Unsupervised Pretraining</h1>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [41]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [42]:
# Collect predictions scanning runs
#predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [43]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,20.0,1.0,58.31,42.53,72.16,11.42
1,H-UNet-BASE-SWTA-T,0,20.0,1.0,61.81,46.37,64.47,10.11
2,H-UNet-HPCA,0,1.0,1.0,45.38,29.96,97.46,15.54
3,H-UNet-HPCA-T,0,1.0,1.0,41.45,26.72,58.05,11.22


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   20.0     1.0     58.31 NaN   42.53 NaN              72.16   
H-UNet-BASE-SWTA-T 20.0     1.0     61.81 NaN   46.37 NaN              64.47   
H-UNet-HPCA        1.0      1.0     45.38 NaN   29.96 NaN              97.46   
H-UNet-HPCA-T      1.0      1.0     41.45 NaN   26.72 NaN              58.05   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   20.0     1.0    NaN                    11.42 NaN  
H-UNet-BASE-SWTA-T 20.0     1.0    NaN                    10.11 NaN  
H-UNet-HPCA        1.0      1.0    NaN                    15.54 NaN  
H-UNet-HPCA-T      1.0      1.0    NaN                    11.22 NaN

<h1>Evaluation - Data regime variations</h1>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

<h2>GlaS Dataset</h2>

In [12]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

REGIMES = ['0.2']       # regimes to be considered
INV_TEMP_GlaS=50          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=glas/unet_base/inv_temp-1').glob('regime-*')),
    #'UNet-256': list(Path(EXP_ROOT + '/experiment=glas/unet_base-256/inv_temp-1').glob('regime-*')),
    #'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT-FROM-LAST': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromLast/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT-FROM-LAST': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromLast/inv_temp-1').glob('regime-*')),
    #'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-FT-FROM-LAST': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromLast/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-T-FT-FROM-LAST': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromLast/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [13]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv', regimes=REGIMES) for k, v in runs.items() for r in v], ignore_index=True)

In [14]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT-FROM-BEST-DICE,0,1.0,0.2,78.85,65.75,20.67,2.67
1,H-UNet-HPCA-FT-FROM-BEST-DICE,1,1.0,0.2,76.29,62.54,26.77,3.77
2,H-UNet-HPCA-FT-FROM-BEST-DICE,2,1.0,0.2,71.30,56.49,41.77,5.86
3,H-UNet-HPCA-FT-FROM-BEST-DICE,3,1.0,0.2,73.87,60.03,36.07,5.38
4,H-UNet-HPCA-FT-FROM-BEST-DICE,4,1.0,0.2,75.75,61.87,24.19,3.40
5,H-UNet-HPCA-FT-FROM-LAST,0,1.0,0.2,75.78,62.14,41.99,5.87
6,H-UNet-HPCA-FT-FROM-LAST,1,1.0,0.2,76.72,63.25,24.03,3.64
7,H-UNet-HPCA-FT-FROM-LAST,2,1.0,0.2,71.49,56.95,34.98,5.19
8,H-UNet-HPCA-FT-FROM-LAST,3,1.0,0.2,74.35,60.29,30.75,4.12
9,H-UNet-HPCA-FT-FROM-LAST,4,1.0,0.2,77.37,64.08,22.66,3.33


Dice           Jaccard  \
                                                   mean       std    mean   
Model                           Inv Temp Regime                             
H-UNet-HPCA-FT-FROM-BEST-DICE   1.0      0.2     75.212  2.819188  61.336   
H-UNet-HPCA-FT-FROM-LAST        1.0      0.2     75.904  1.921164  62.311   
H-UNet-HPCA-T-FT-FROM-BEST-DICE 1.0      0.2     73.584  3.861558  59.276   
H-UNet-HPCA-T-FT-FROM-LAST      1.0      0.2     75.037  1.843150  61.085   
H-UNet-SWTA-FT-FROM-BEST-DICE   50.0     0.2     76.638  2.727567  63.198   
H-UNet-SWTA-FT-FROM-LAST        50.0     0.2     77.704  1.469874  64.572   
H-UNet-SWTA-T-FT-FROM-BEST-DICE 50.0     0.2     75.836  2.129878  62.096   
H-UNet-SWTA-T-FT-FROM-LAST      50.0     0.2     76.680  2.810952  63.258   
UNet                            1.0      0.2     78.462  2.753385  65.684   

                                                          Hausdorff Distance  \
                                                      std               mean   
Model                           Inv Temp Regime                                
H-UNet-HPCA-FT-FROM-BEST-DICE   1.0      0.2     3.406153             29.894   
H-UNet-HPCA-FT-FROM-LAST        1.0      0.2     2.407897             30.350   
H-UNet-HPCA-T-FT-FROM-BEST-DICE 1.0      0.2     4.586794             26.504   
H-UNet-HPCA-T-FT-FROM-LAST      1.0      0.2     2.342061             28.628   
H-UNet-SWTA-FT-FROM-BEST-DICE   50.0     0.2     3.382716             24.784   
H-UNet-SWTA-FT-FROM-LAST        50.0     0.2     1.894695             26.066   
H-UNet-SWTA-T-FT-FROM-BEST-DICE 50.0     0.2     2.672570             26.124   
H-UNet-SWTA-T-FT-FROM-LAST      50.0     0.2     3.500624             26.092   
UNet                            1.0      0.2     3.510575             27.049   

                                                           \
                                                      std   
Model                           Inv Temp Regime             
H-UNet-HPCA-FT-FROM-BEST-DICE   1.0      0.2     8.754421   
H-UNet-HPCA-FT-FROM-LAST        1.0      0.2     6.096055   
H-UNet-HPCA-T-FT-FROM-BEST-DICE 1.0      0.2     6.583823   
H-UNet-HPCA-T-FT-FROM-LAST      1.0      0.2     5.648588   
H-UNet-SWTA-FT-FROM-BEST-DICE   50.0     0.2     4.302357   
H-UNet-SWTA-FT-FROM-LAST        50.0     0.2     7.051718   
H-UNet-SWTA-T-FT-FROM-BEST-DICE 50.0     0.2     5.233529   
H-UNet-SWTA-T-FT-FROM-LAST      50.0     0.2     3.938016   
UNet                            1.0      0.2     2.833017   

                                                Average Surface Distance  \
                                                                    mean   
Model                           Inv Temp Regime                            
H-UNet-HPCA-FT-FROM-BEST-DICE   1.0      0.2                       4.216   
H-UNet-HPCA-FT-FROM-LAST        1.0      0.2                       4.284   
H-UNet-HPCA-T-FT-FROM-BEST-DICE 1.0      0.2                       3.750   
H-UNet-HPCA-T-FT-FROM-LAST      1.0      0.2                       4.038   
H-UNet-SWTA-FT-FROM-BEST-DICE   50.0     0.2                       3.274   
H-UNet-SWTA-FT-FROM-LAST        50.0     0.2                       3.670   
H-UNet-SWTA-T-FT-FROM-BEST-DICE 50.0     0.2                       3.650   
H-UNet-SWTA-T-FT-FROM-LAST      50.0     0.2                       3.662   
UNet                            1.0      0.2                       3.807   

                                                           
                                                      std  
Model                           Inv Temp Regime            
H-UNet-HPCA-FT-FROM-BEST-DICE   1.0      0.2     1.352084  
H-UNet-HPCA-FT-FROM-LAST        1.0      0.2     0.851002  
H-UNet-HPCA-T-FT-FROM-BEST-DICE 1.0      0.2     0.949184  
H-UNet-HPCA-T-FT-FROM-LAST      1.0      0.2     0.817663  
H-UNet-SWTA-FT-FROM-BEST-DICE   50.0     0.2     0.493336  
H-UNet-SWTA-FT-FROM-LAST        50.0   